## 9.5_DQN_for_Atari_Games

In [1]:
import gym
import sys
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.misc import imresize
import time
from skimage.transform import resize

In [2]:
MAX_EXPERIENCES = 500000
MIN_EXPERIENCES = 50000
TARGET_UPDATE_PERIOD = 10000
IM_SIZE = 80
K = 4

### preprocess function

In [3]:
def preprocess(img):
    img_temp = img[31:195] # choose the important area of the image
    img_temp = img_temp.mean(axis=2) # Convert to Grayscale
    
    # Downsample image using nearest neighbour interpolation
    img_temp = resize(img_temp, (IM_SIZE,IM_SIZE))
    return img_temp

# testing for the above function
def test_preprocess():
    env = gym.make("Breakout-v0")
    observation = env.reset()
    img = preprocess(observation)

    plt.imshow(observation)
    plt.show()
    print(observation.shape)

    plt.imshow(img)
    plt.show()
    print(img.shape)
    env.close()

# test_preprocess()

In [4]:
def update_state(state, obs):
    obs_small = preprocess(obs)
    return np.append(state[1:], np.expand_dims(obs_small, 0), axis = 0)

In [5]:
class DQN:
    def __init__(self, K, scope, save_path = 'models/atari.ckpt'):
        self.K = K 
        self.scope = scope
        self.save_path = save_path
        
#         tf.reset_default_graph()
        with tf.variable_scope(scope):
            
            # inputs and targets
            self.X = tf.placeholder(tf.float32, shape=(None, 4, IM_SIZE, IM_SIZE), name='X')
            # tensorflow convolution needs the order to be:
            # (num_samples, height, width, "color")
            # so we need to transpose later
            self.G = tf.placeholder(tf.float32, shape=(None,), name='G')
            self.actions = tf.placeholder(tf.int32, shape=(None,), name='actions')
            
            # calculate output and cost
            # convolutional layers
            Z = self.X / 255.0
            Z = tf.transpose(Z, [0,2,3,1])
            # tf.contrib.layers.conv2d
            # input:[batch_size] + input_spatial_shape + [in_channels]
            # (num_outputs, kernel_size, stride=1)
            cnn1 = tf.contrib.layers.conv2d(Z, 32, 8, 4, activation_fn=tf.nn.relu)
            cnn2 = tf.contrib.layers.conv2d(cnn1, 64, 4, 2, activation_fn=tf.nn.relu)
            cnn3 = tf.contrib.layers.conv2d(cnn2, 64, 3, 1, activation_fn=tf.nn.relu)
            
            # fully connected layers
            fc0 = tf.contrib.layers.flatten(cnn3)
            fc1 = tf.contrib.layers.fully_connected(fc0, 512)
            
            # final output layer
            self.predict_op = tf.contrib.layers.fully_connected(fc1,K)
            selected_action_values = tf.reduce_sum(
                self.predict_op * tf.one_hot(self.actions, K),
                reduction_indices = [1]
            )
            # tf.reduce_sum : get the sum along an axis
            
            self.cost = tf.reduce_mean(tf.square(self.G - selected_action_values))
            self.train_op = tf.train.RMSPropOptimizer(learning_rate=0.00025, decay=0.9,
                                                      momentum=0.0, epsilon=1e-6
                                                     ).minimize(self.cost)
    
    def set_session(self,session):
        self.session = session
        
    def predict(self, states):
        return self.session.run(self.predict_op, feed_dict={self.X:states})
        
    def update(self, states, actions, targets):
        c, _ = self.session.run(
            [self.cost, self.train_op],
            feed_dict={
                self.X:states,
                self.G:targets,
                self.actions:actions
            }
        )
        return c
    
    def sample_action(self, x, eps):
        """implements epsilon greedy algorithm"""
        if np.random.random() < eps:
            return np.random.choice(self.K)
        else:
            return np.argmax(self.predict([x])[0])
    
    def load(self):
        self.saver = tf.train.Saver(tf.global_variables())
        load_was_success = True
        try:
            save_dir = '/'.join(self.save_path.split('/')[:-1])
            ckpt = tf.train.get_checkpoint_state(save_dir)
            load_path = ckpt.model_checkpoint_path
            self.saver.restore(self.session, load_path)
        except:
            print('no saved model to load. starting new session')
            load_was_success = False
        else:
            print("loaded model: {}".format(load_path))
            saver = tf.train.Saver(tf.global_variables())
            episode_number = int(load_path.split('-')[-1])
            
    def save(self, n):
        self.saver.save(self.session, self.save_path, global_step=n)
        print("SAVED MODEL #{}".format(n))
        
    def copy_from(self, other):
        mine = [t for t in tf.trainable_variables() 
                if t.name.startswith(self.scope)]
        mine = sorted(mine, key=lambda v: v.name)
        others = [t for t in tf.trainable_variables() 
                if t.name.startswith(other.scope)]
        others = sorted(others, key=lambda v: v.name)
        
        ops = []
        for p, q in zip(mine, others):
            actual = self.session.run(q)
            op = p.assign(actual)
            ops.append(op)
        
        self.session.run(ops)

In [6]:
def learn(model,targets_model, experience_replay_buffer, gamma, batch_size):
    # sample experiences
    samples = random.sample(experience_replay_buffer, batch_size)
    states, actions, rewards, next_states, dones = map(np.array, zip(*samples))

    # calculate targets
    next_Qs = target_model.predict(next_states)
    next_Q = np.amax(next_Qs, axis=1)
    targets = rewards + np.invert(dones).astype(np.float32) * gamma * next_Q

#     print(states.shape)
#     print(actions.shape)
#     print(targets.shape)
    
    # Update model
    loss = model.update(states, actions, targets)
    return loss

In [7]:
# main function
# hyperparameters
gamma = 0.99
batch_sz = 32
num_episodes = 500
total_t = 0
experience_replay_buffer = []
episode_rewards = np.zeros(num_episodes)
last_100_avgs = []

# epsilon for Epsilon Greedy Algorithm
epsilon = 1.0
epsilon_min = 0.1
epsilon_change = (epsilon - epsilon_min) / 500000

# Create Atari Environment
env = gym.envs.make("Breakout-v0")

In [ ]:
# Create original and target Networks
model = DQN(K=K, scope="model")
target_model = DQN(K=K, scope="target_model")

with tf.Session() as sess:
    model.set_session(sess)
    target_model.set_session(sess)
    sess.run(tf.global_variables_initializer())
    model.load()
    
    print("Filling experience replay buffer ...")
    obs = env.reset()
    obs_small = preprocess(obs)
    state = np.stack([obs_small] * 4, axis = 0)
    
    # Fill experience replay buffer
    for i in range(MIN_EXPERIENCES):
        action = np.random.randint(0,K)
        obs, reward, done, _ = env.step(action)
        next_state = update_state(state, obs)
        
        experience_replay_buffer.append((state, action, reward, next_state, done))
        
        if done:
            obs = env.reset()
            obs_small = preprocess(obs)
            state = np.stack([obs_small] * 4, axis = 0)
        
        else:
            state = next_state
    
    print("Playing and learning ...")
    # Play a number of episodes and learn
    for i in range(num_episodes):
        t0 = datetime.now()
        
        # Reset the environment
        obs = env.reset()
        obs_small = preprocess(obs)
        state = np.stack([obs_small]*4,axis=0)
        assert (state.shape == (4,80,80))
        loss = None
        
        total_time_training = 0
        num_steps_in_episode = 0
        episode_reward = 0
        
        done = False
        while not done:
            # Update target network
            if total_t % TARGET_UPDATE_PERIOD == 0:
                target_model.copy_from(model)
                print("Copied model parameters to target network. total_t = %s, period = %s"
                     % (total_t, TARGET_UPDATE_PERIOD))
            
            # Take action
            action = model.sample_action(state, epsilon)
            obs, reward, done, _ = env.step(action)
            obs_small = preprocess(obs)
            next_state = np.append(state[1:], np.expand_dims(obs_small, 0), axis=0)
            
            episode_reward += reward
            
            # Remove oldest experience if replay buffer is full
            if len(experience_replay_buffer) == MAX_EXPERIENCES:
                experience_replay_buffer.pop(0)
            # Save the recent experience
            experience_replay_buffer.append((state,action,reward,next_state,done))
            
            # Train the model and keep measure of time
            t0_2 = datetime.now()
            loss = learn(model, target_model, experience_replay_buffer, gamma, batch_sz)
            dt = datetime.now() - t0_2
            
            total_time_training += dt.total_seconds()
            num_steps_in_episode += 1
            
            state = next_state
            total_t += 1
            
            epsilon = max(epsilon - epsilon_change, epsilon_min)
        
        duration = datetime.now() - t0
        
        episode_rewards[i] = episode_reward
        time_per_step = total_time_training / num_steps_in_episode
        
        last_100_avg = episode_rewards[max(0,i-100):i+1].mean()
        last_100_avgs.append(last_100_avg)
        print("Episode:", i, "Duration:",duration, "Num steps:", num_steps_in_episode,
              "Reward:", episode_reward, "Training time per step:", "%.3f" % time_per_step,
              "Avg Reward (Last 100):", "%.3f" % last_100_avg, "Epsilon:", "%.3f" % epsilon
        )
        
        if i % 50 == 0:
            model.save(i)
        sys.stdout.flush()
    
    # Plots
    plt.plot(last_100_avgs)
    plt.xlabel('episodes')
    plt.ylabel('Average Rewards')
    plt.show()
    env.close()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.cast instead.
no saved model to load. starting new session
Filling experience replay buffer ...


/Applications/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Applications/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Playing and learning ...
Copied model parameters to target network. total_t = 0, period = 10000
Episode: 0 Duration: 0:00:18.374611 Num steps: 183 Reward: 0.0 Training time per step: 0.094 Avg Reward (Last 100): 0.000 Epsilon: 1.000
SAVED MODEL #0
Episode: 1 Duration: 0:00:21.774477 Num steps: 220 Reward: 1.0 Training time per step: 0.096 Avg Reward (Last 100): 0.500 Epsilon: 0.999
Episode: 2 Duration: 0:00:28.082412 Num steps: 298 Reward: 2.0 Training time per step: 0.092 Avg Reward (Last 100): 1.000 Epsilon: 0.999
Episode: 3 Duration: 0:00:20.463376 Num steps: 210 Reward: 1.0 Training time per step: 0.095 Avg Reward (Last 100): 1.000 Epsilon: 0.998
Episode: 4 Duration: 0:00:24.152785 Num steps: 252 Reward: 2.0 Training time per step: 0.094 Avg Reward (Last 100): 1.200 Epsilon: 0.998
Episode: 5 Duration: 0:00:20.457821 Num steps: 220 Reward: 1.0 Training time per step: 0.091 Avg Reward (Last 100): 1.167 Epsilon: 0.998
Episode: 6 Duration: 0:00:20.866521 Num steps: 220 Reward: 1.0 Trai

In [27]:
env.close()